<a href="https://colab.research.google.com/github/auzaluis/2024-1_UTEPSA_BigData_AnalisisDeDatos/blob/main/Python/01_script_ETL_personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 1: Carga de datos

## **Importando base de datos**

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [ ]:
# API Client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread Authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url_personalidad = "https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing"
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet("Respuestas de formulario 1").get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo en un df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns = sheets[0])

## Inspección data frame

In [ ]:
type(df)

In [ ]:
# Visualizando las primeras filas
df.head()

In [ ]:
# Mirada general
df.info()

In [ ]:
# Tipo de datos
df["Sexo"].dtype

In [ ]:
# Número de filas y columnas
df.shape

In [ ]:
# Número de filas
len(df)

In [ ]:
df.columns

In [ ]:
len(df.columns)

# **Tema 2: Transformación de datos**

## **Valores perdidos**

### **Identificación de los NAs**

In [ ]:
# Los NAs en realidad son strings vacíos
import numpy as np
df.replace("", np.nan, inplace = True)

In [ ]:
df['Escribe tu edad exacta'].isna().value_counts()

### **Omitiendo**



In [ ]:
df['Escribe tu edad exacta'].dtype

In [ ]:
df['Escribe tu edad exacta'] = pd.to_numeric(df['Escribe tu edad exacta'], errors = 'coerce')

In [ ]:
df['Escribe tu edad exacta'].dtype

In [ ]:
df['Escribe tu edad exacta'].mean()

In [ ]:
edad_promedio = df['Escribe tu edad exacta'].mean()

### **Reemplazo por la media**

In [ ]:
# creando df2
df2 = df.copy()

In [ ]:
# reemplazo por la media
df2["edad2"] = df2["Escribe tu edad exacta"].fillna(edad_promedio)

**Reubicando la columna edad2**

In [ ]:
# Colocar los nombres de las columnas en una lista
lista_columnas = df2.columns.to_list()

In [ ]:
# Quitar temporalmente edad2 de la lista
lista_columnas.remove("edad2")

In [ ]:
# obteniendo el índice de la columna 'after'
lista_columnas.index("Escribe tu edad exacta")

In [ ]:
# insertando 'edad2' después de 'Escribe tu edad exacta'
lista_columnas.insert(lista_columnas.index("Escribe tu edad exacta") + 1, "edad2")

In [ ]:
# Reordenando y guadando en df2
df2 = df2[lista_columnas]

**Emulando relocate() de R en Python**

In [ ]:
def relocate(df, columna, after):
  lista_columnas = df.columns.to_list()
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [ ]:
relocate(
    df = df2,
    columna = "edad2",
    after = "Escribe tu edad exacta"
)

### **Eliminar la fila completa**

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.shape

## **Estandarización de variables**

### **Normalización**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# instanciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
# normalizando
normalizador.fit_transform(df2[["edad2"]])

In [ ]:
# Creando df3
df3 = df2.copy()

In [ ]:
df3["edadZ"] = normalizador.fit_transform(df2[["edad2"]])

In [ ]:
# Reubicando edadZ
df3 = relocate(
    df = df3,
    columna = "edadZ",
    after = "edad2"
)

### **Rango**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [ ]:
rango.fit_transform(df3[["edad2"]])

In [ ]:
df3["edadRango"] = rango.fit_transform(df3[["edad2"]])

In [ ]:
df3 = relocate(
    df = df3,
    columna = "edadRango",
    after = "edadZ"
)

In [ ]:
df3.head()

## **Agrupaciones**

### **Numéricas**

In [ ]:
cortes = [-float("inf"), 18, 21, float("inf")]
etiquetas = ["18 o menos", "19 a 21", "Más de 21"]

In [ ]:
# Creando columna edadGR
df3["edadGR"] = pd.cut(
    df3["edad2"],
    bins = cortes,
    labels = etiquetas
)

In [ ]:
# comparando
pd.DataFrame({
    "original": df3["edad2"],
    "edadGR": df3["edadGR"]
})

### **Categóricas**

In [ ]:
# índices del df3
df3.info()

In [ ]:
# identificando los niveles
pd.unique(df3.iloc[:,7])

In [ ]:
df3.iloc[:,7].value_counts()

In [ ]:
# prueba lógica top 2 box
df3.iloc[:,7].isin(["Totalmente verdadero", "Un poco verdadero"]).value_counts()

**Dummitización**

In [ ]:
# función condicional normal
def top2box(x):
  if x in ["Totalmente verdadero", "Un poco verdadero"]:
    return 1
  else:
    return 0

In [ ]:
# aplicando la función
df3.iloc[:,7].apply(top2box)

In [ ]:
# frecuencias
df3.iloc[:,7].apply(top2box).value_counts()

**Funciones lambda**

In [ ]:
df3.iloc[:,7].apply(lambda x: 1 if x in ["Totalmente verdadero", "Un poco verdadero"] else 0).value_counts()

**Escala ordinal**

In [ ]:
import numpy as np

In [ ]:
# niveles
pd.unique(df3.iloc[:,7])

In [ ]:
condiciones = [
    df3.iloc[:,7] == "Totalmente falso",
    df3.iloc[:,7] == "Un poco falso",
    df3.iloc[:,7] == "No lo sé",
    df3.iloc[:,7] == "Un poco verdadero",
    df3.iloc[:,7] == "Totalmente verdadero"
]

opciones = [1, 2, 3, 4, 5]

In [ ]:
ordinal = np.select(
    condlist = condiciones,
    choicelist = opciones,
    default = np.nan
)

In [ ]:
# como se trata de un objeto de numpy, el método es distinto
np.unique(ordinal, return_counts = True)

In [ ]:
# si quisieras usar value_counts()...
pd.Series(ordinal).value_counts()

**Bucles**

Forma convencional

In [ ]:
# creando df4
df4 = df3.copy()

In [ ]:
df4.columns

In [ ]:
# crear un vector vacío
frases = []

# creando bucle para llenar vector vacío
for col in df4.columns:
  if col.startswith("Según tu"):
    frases.append(col)

In [ ]:
frases

In [ ]:
# creando otro bucle, esta vez para dummitizar
for var in frases:
  df4[var] = df4[var].apply(top2box)